# Bot Players: Assign Equipment

In [1]:
import os, sys, random, json, datetime
import pandas as PD

In [2]:
BACKEND_PATH = os.path.dirname(os.path.dirname(os.getcwd()))
APPPROJ_PATH = os.path.join(BACKEND_PATH, 'app_proj')
sys.path.append(APPPROJ_PATH)
import notebooks as NT
print([x for x in dir(NT) if x[0]!='_' and len(x)>8])

['DataframeToDicts', 'GetRandom', 'NOTEBOOK_ENV']


In [3]:
import django
os.environ['DEPLOYMENT_ENV'] = NT.NOTEBOOK_ENV
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = 'True'
MODULE_PATH = os.path.dirname(os.getcwd())
os.chdir(os.path.dirname(MODULE_PATH))
django.setup()

env: DEV


In [4]:
from django.utils import timezone

In [5]:
import database as DB
print([x for x in dir(DB) if x[0]!='_' and len(x)>8])

['BaseManager', 'DeleteTable', 'GetNativeTableCounts', 'GetTableCounts', 'GetTableDictionary', 'InsertBulk', 'InsertSingle']


In [6]:
import django.db.models as JM
from django.contrib.auth import get_user_model

In [7]:
import emporium.models as EM

import engine.models as GM
import engine.logic.resource as RS
import engine.logic.simulation as SM

In [8]:
%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

### Guild Properties

In [9]:
userMd = get_user_model().objects.filter(user_name='Admin')[0]
userMd

<User: admin@thieves-guild.com>

In [10]:
guild = GM.Guild.objects.GetOrNone(UserFK=userMd, Name='123')
PD.DataFrame([guild.__dict__]).drop(['id', '_state'], axis=1, errors='ignore')

,UserFK_id,Name,KeepLevel,TotalPower,VaultGold,VaultGems,Selected,LastPlayed,CreateDate
0,1,123,1,0,0,0,True,2023-08-11,2023-08-11


In [11]:
thieves = GM.ThiefInGuild.objects.filter(GuildFK=guild).values()
PD.DataFrame(thieves).drop(['id',
        'Name', 'BasePower', 'Experience', 
        'Position', 
        'BaseAgi', 'BaseCun', 'BaseMig', 'BaseEnd', 
        'TrainedAgi', 'TrainedCun', 'TrainedMig', 'TrainedEnd', 
        'Sabotage', 'Perceive', 'Traverse',
        ], axis=1, errors='ignore')

,GuildFK_id,Class,Stars,Level,Power,Agility,Cunning,Might,Endurance,Health,Attack,Damage,Defense,Wounds,Cooldown
0,15,Burglar,1,1,70,5,0,0,0,70,5,11,13,None,None
1,15,Scoundrel,1,1,70,0,5,0,0,70,2,15,12,None,None
2,15,Ruffian,1,1,70,0,0,5,0,70,3,10,16,None,None


In [12]:
# guildItems = GM.ItemInGuild.objects.filter(GuildFK=guild).values()
# PD.DataFrame(guildItems).drop(['id'], axis=1, errors='ignore')

In [13]:
report = SM.EquipmentReport(guild)
PD.DataFrame(report)

,Class,Power,Equipment
0,Burglar,70,"weapon, armor, head"
1,Scoundrel,70,"weapon, armor, hands"
2,Ruffian,70,"weapon, armor, feet"
3,Vault,N/A,{}


In [14]:
SM.RemoveItems(guild)

report = SM.EquipmentReport(guild)
PD.DataFrame(report)

,Class,Power,Equipment
0,Burglar,30,
1,Scoundrel,30,
2,Ruffian,30,
3,Vault,N/A,"{""weapon"": 3, ""armor"": 3, ""head"": 1, ""hands"": ..."


In [15]:
SM.AssignWargear(guild, 'weapon', 'Burglar')
SM.AssignWargear(guild, 'weapon', 'Scoundrel')
SM.AssignWargear(guild, 'weapon', 'Ruffian')

SM.AssignWargear(guild, 'armor', 'Burglar')
SM.AssignWargear(guild, 'armor', 'Scoundrel')
SM.AssignWargear(guild, 'armor', 'Ruffian')

In [16]:
report = SM.EquipmentReport(guild)
PD.DataFrame(report)

,Class,Power,Equipment
0,Burglar,60,"weapon, armor"
1,Scoundrel,60,"weapon, armor"
2,Ruffian,60,"weapon, armor"
3,Vault,N/A,"{""head"": 1, ""hands"": 1, ""feet"": 1}"


In [17]:
roll = random.randint(1,3)
if roll == 1:
    SM.AssignAccessory(guild, 'head')
    SM.AssignAccessory(guild, 'hands')
    SM.AssignAccessory(guild, 'feet')
if roll == 2:
    SM.AssignAccessory(guild, 'hands')
    SM.AssignAccessory(guild, 'feet')
    SM.AssignAccessory(guild, 'head')
if roll == 3:
    SM.AssignAccessory(guild, 'feet')
    SM.AssignAccessory(guild, 'head')
    SM.AssignAccessory(guild, 'hands')

In [18]:
report = SM.EquipmentReport(guild)
PD.DataFrame(report)

,Class,Power,Equipment
0,Burglar,70,"weapon, armor, hands"
1,Scoundrel,70,"weapon, armor, feet"
2,Ruffian,70,"weapon, armor, head"
3,Vault,N/A,{}
